In [ ]:
!pip --quiet install huggingface_hub

In [ ]:
!huggingface-cli login

In [3]:
# Kiểm tra xem colab đã sử dụng GPU chưa
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# Click vào biểu tượng mount Drive ở thanh điều hướng bên trái để kết nối với drive của mình
# Đôi lúc không có biểu tượng đó thì phải chạy hai lệnh dưới để kết nối

from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
# Nên tạo một thư mục chứa code và model ở drive của mình
# Ở đây mình tạo trước một thư mục chứa code và model ở địa chỉ /content/drive/MyDrive/qag_pegasus/
# Cần di chuyển đến thư mục đó, từ đoạn này trở đi, mọi thác tác sẽ được thực hiện ở thư mục đó

In [4]:
cd /content/drive/MyDrive/qag_pegasus/

/content/drive/MyDrive/VNPT/qag_pegasus


**Cấu trúc thư mục qag_pegasus ở goolde driver của mình sẽ như sau:**
```
qag_pegasus/
    - mounts/        #Thư mục này để chứa mọi thứ như dữ liệu, model, ...
        - models/
            - pegasus-xsum/          #Thư mục cache để lưu pegasus-xsum
            - qag_pegasus_mrl_model/ #thư mục chứ model sau khi finetune xong
        - data/                      #Thư mục chứa các file dữ liệu
            - MCQ_Squad_MRL.csv
    - qag_pegasus/                   #Thư mục chứa các file code.py
        - __init__.py
        - min_ref_loss_model.py
        - run_train.py
        - training_agruments.py
        - mydatasets.py
    - requirements.txt
    - setup.py
```

In [ ]:
# Cài đặt các thư viện cần thiết và cài đặt project package
!pip --quiet install -r requirements.txt
!pip --quiet install -e .

---
### Training

In [ ]:
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
!python qag_pegasus/run_train.py \
    --model_name_or_path google/pegasus-xsum \
    --cache_dir mounts/models/pegasus-xsum \
    --output_dir mounts/models/qag_pegasus_mrl_model \
    --train_file mounts/data/MCQ_Squad_MRL.csv \
    --push_to_hub=True \
    --source_max_token_len 256 \
    --target_max_token_len 64 \
    --num_train_epochs 1 \
    --per_device_train_batch_size 2 \
    --save_steps 100000000 \
    --logging_steps 100 \
    --save_total_limit 1 \
    --warmup_steps=150 \
    --weight_decay=0.1 \
    --learning_rate=0.00005



/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 12188
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 6094
{'loss': 3.9153, 'learning_rate': 3.3333333333333335e-05, 'epoch': 0.02}
{'loss': 2.5025, 'learning_rate': 4.957940780619112e-05, 'epoch': 0.03}
{'loss': 1.884, 'learning_rate': 4.8738223418573355e-05, 'epoch': 0.05}
{'loss': 1.8276, 'learning_rate': 4.789703903095559e-05, 'epoch': 0.07}
{'loss': 1.7711, 'learning_rate': 4.705585464333782e-05, 'epoch': 0.08}
{'loss': 1.5458, 'learning_rate': 4.6214670255720054e-05, 'epoch': 0.1}
{'loss': 

---
### Test

In [ ]:
from qag_pegasus import QAGPegasus

qag = QAGPegasus(model_name_or_path="mounts/models/qag_pegasus_mrl_model")

In [ ]:
context = '''President Joe Biden ordered airstrikes against Iranian-backed groups in Syria on Tuesday, a little over a week after a number of rockets struck near a military base in northeastern Syria housing US troops. The airstrikes conducted by the US military targeted Iranian-backed groups in Deir ez-Zor Syria, US Central Command said in a statement. The strikes targeted "infrastructure facilities used by groups affiliated with Iran's Islamic Revolutionary Guard Corps," Col. Joe Buccino, a spokesman for CENTCOM, said in the statement. "At President Biden's direction, US military forces conducted precision airstrikes in Deir ez-Zor Syria today. These precision strikes are intended to defend and protect US forces from attacks like the ones on August 15 against US personnel by Iran-backed groups," he said, referring to last week's attacks on the Green Village base near the Iraqi border. The incident did not result in damage or injuries.
'''

In [ ]:
len(context.split(' '))

In [ ]:
outputs = qag.generate_qa(context,
                          num_return_sequences=4,
                          max_length=64,
                          do_sample=True,
                          top_k=6,
                          top_p=0.9,
                          temperature=0.7,
                          no_repeat_ngram_size=2,
                          early_stopping=True
                          )
for sequence in outputs:
    print(sequence)
